# Cyclistic Bike Share Case Study

### Business Task

Cyclistic - The Bike sharing company aims to grow further by maximizing its annual memberships. This can be done by converting the casual riders to annual members. To accomplish this, the marketing team needs to design focused marketing strategies. Hence, the task is to analyze the company’s user data to understand how the casual riders use the bike-sharing service differently than the annual members.

### Data Information

The data used for the analysis is Cyclistic’s historical trip data for 12 months, ranging from November 2020 to October 2021.
1. It is distributed under a [license](https://www.divvybikes.com/data-license-agreement)
2. The data is stored externally on the [cloud](https://divvy-tripdata.s3.amazonaws.com/index.html)
3. There is no Personally Identifiable Information (PII).
4. First-Party data, hence, original, and reliable.
5. The data is downloaded and stored appropriately in a folder.


### Data Processing

This is the stage to check the data to ensure integrity, completeness, correctness, relevance; and to perform data cleaning.

### Tools:

1.	Spreadsheets – Perform initial data inspection and get an overview.
2.	RStudio – Develop script to clean, transform, organize, and summarize the data.
3.	Tableau – Create Visualizations and Dashboard for final presentation.


### EXPLORATORY DATA ANALYSIS 

In [ ]:
## IMPORT LIBRARIES

library(tidyverse)  #tidyverse includes core packages like ggplot2 and readr which are helpful to wrangle data 
library(lubridate)  #helps wrangle date attributes
library(data.table) #helps to write data into csv file
library(skimr)  #to provide summary statistics about variables in data frames, tibbles, data tables and vectors
library(dplyr)  #grammar of data manipulation, providing a consistent set of verbs
library(janitor)  #to examine and clean dirty data
library(modeest)  #provides estimators of the mode of univariate data or univariate distributions
library(ggplot2)  #helps visualize data

#### Step-1 : Collect Data

In [ ]:
## READ ALL THE DATA STORED IN CSV FILES

d1 <- read.csv("202011-divvy-tripdata.csv")
d2 <- read.csv("202012-divvy-tripdata.csv")
d3 <- read.csv("202101-divvy-tripdata.csv")
d4 <- read.csv("202102-divvy-tripdata.csv")
d5 <- read.csv("202103-divvy-tripdata.csv")
d6 <- read.csv("202104-divvy-tripdata.csv")
d7 <- read.csv("202105-divvy-tripdata.csv")
d8 <- read.csv("202106-divvy-tripdata.csv")
d9 <- read.csv("202107-divvy-tripdata.csv")
d10 <- read.csv("202108-divvy-tripdata.csv")
d11 <- read.csv("202109-divvy-tripdata.csv")
d12 <- read.csv("202110-divvy-tripdata.csv")

#### Step-2 : Data Wrangling

In [ ]:
## CHECK COLUMN NAMES FOR CONSISTENCY

colnames(d1)
colnames(d2)
colnames(d3)
colnames(d4)
colnames(d5)
colnames(d6)
colnames(d7)
colnames(d8)
colnames(d9)
colnames(d10)
colnames(d11)
colnames(d12)


### Metrics

*After comparing column names, we can infer that there  are 13 variables/columns available, below is the metadata:*

**ride_id** : Unique id of each ride trip

**rideable_type** : type of bicycle ridden, split between 3 categories - classic, docked and electric

**started_at** : date and time of the start of the trip

**ended_at** : date and time of the end of the trip

**start_station_name** : Start station name

**start_station_id** : Start station id

**end_station_name** : End station name

**end_station_id** : End station id

**start_lat** : latitude of the start location

**start_lng** : longitude of the start location

**end_lat** : latitude of the end location

**end_lng** : longitude of the end location

**member_casual** : type of membership, either casual or member

In [ ]:
## CHECK THE STRUCTURE OF DATA

str(d1)
str(d2)
str(d3)
str(d4)
str(d5)
str(d6)
str(d7)
str(d8)
str(d9)
str(d10)
str(d11)
str(d12)

#### The columns - 'end_station_id' and 'start_station_id' have numeric data types. 
#### But, they are supposed to be character type as they represent Station ID's.
#### Hence, we'll change the type of the two columns in data of 'd1' dataframe.
#### Other dataframes have above mentioned columns as character type so we need not worry about them.

In [ ]:
## CHANGE DATASET D1'S END_STATION_ID & START_STATION_ID FROM - NUM TO CHAR
## OTHERS HAVE CHAR VALUES FOR STATION IDS

d1 <- mutate(d1, end_station_id = as.character(end_station_id), start_station_id = as.character(start_station_id))


#### Now, we'll compile the data for each month into one dataframe.

In [ ]:
## COMPILE ALL DATASETS TO ONE DATASET - CYDATA

cydata <- rbind(d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12)
summary(cydata)
head(cydata)
str(cydata)


#### Now, we'll check out the number of rows with null values and remove them.

In [ ]:
## CHECK THE AMOUNT OF NULL VALUES IN ROWS OF CYDATA
sum(is.na(cydata))

## REMOVE NULL VALUES AND STORE THE REST IN CYDATA_CLEANED
cydata_cleaned <- drop_na(cydata)
sum(is.na(cydata_cleaned))


#### Since, Ride ID's are supposed to be unique, we need to remove duplicate Ride ID's.

In [ ]:
## CHECK THE NUMBER OF DUPLICATE RIDE IDS
sum(duplicated(cydata_cleaned$ride_id))

## REMOVE DUPLICATE RIDE IDS
cydata_cleaned <- cydata_cleaned%>%
  filter(!duplicated(cydata_cleaned$ride_id))

## CHECK IF DUPLICATION REMOVAL SUCCEEDED
sum(duplicated(cydata_cleaned$ride_id))


#### We need the 'started_at' and 'ended_at' columns to be of datetime type in order to analyse the data properly.


In [ ]:
## CONVERT CYDATA_CLEANED--STARTED_AT & ENDED_AT COLUMNS FROM CHAR TO DATETIME

cydata_cleaned$started_at<- as.POSIXct(cydata_cleaned$started_at,format = "%Y-%m-%d %H:%M:%S",tz="EST")
cydata_cleaned$ended_at<- as.POSIXct(cydata_cleaned$ended_at,format = "%Y-%m-%d %H:%M:%S",tz="EST")


#### We'll find the ride length for each ride by finding the difference between the start and end time.

In [ ]:
## GET TIME DIFFERENCE OF STARTED_AT & ENDED_AT AND CREATE RIDE_LENGTH COLUMN
cydata_cleaned$ride_length <- difftime(cydata_cleaned$ended_at,cydata_cleaned$started_at)


#### Ride length cannot be a negative values, so we'll remove them. 

In [ ]:
## REMOVE NEGATIVE VALUES FROM RIDE_LENGTH
cydata_cleaned <- cydata_cleaned%>%
  filter(ride_length>=0)

## CHECK IF THE REMOVAL PROCESS SUCCEEDED
sum(cydata_cleaned$ride_length<0)


#### We can extract the day number, month and year for analysing.

In [ ]:
## EXTRACT DAY (NUMBER) FROM STARTED_AT (FOR FUTURE ANALYSIS)
cydata_cleaned$day <- format(as.Date(cydata_cleaned$started_at),"%d")


## EXTRACT MONTH FROM STARTED_AT (FOR FUTURE ANALYSIS)
cydata_cleaned$month <- format(as.Date(cydata_cleaned$started_at),"%m")


## EXTRACT YEAR FROM STARTED_AT (FOR FUTURE ANALYSIS)
cydata_cleaned$year <- format(as.Date(cydata_cleaned$started_at),"%Y")


#### We can also extract the date, hour and day of week for future analysis.

In [ ]:
## DATE COLUMN CREATED 
cydata_cleaned$date <- as.Date((cydata_cleaned$started_at),format="%Y-%m-%d")

## EXTRACT HOUR OF TIME FROM STARTED_AT (FOR FUTURE ANALYSIS)
cydata_cleaned$hour<- format(as.POSIXlt(cydata_cleaned$started_at),format="%H")

## SET DAY_OF_WEEK 
cydata_cleaned$day_of_week <- format(as.Date(cydata_cleaned$started_at),"%A")

#### Checking the station names to find rides when the bike was being brought for maintenance and removing them.

In [ ]:
## CHECK UNIQUE STATION NAMES 
sort(unique(cydata_cleaned$start_station_name))
sort(unique(cydata_cleaned$end_station_name))


## REMOVE BIKE STATION SERVICING RIDES
cydata_cleaned <- cydata_cleaned%>%
  filter(cydata_cleaned$start_station_name!="HUBBARD ST BIKE CHECKING (LBS-WH-TEST)" & cydata_cleaned$end_station_name!="HUBBARD ST BIKE CHECKING (LBS-WH-TEST)")

cydata_cleaned <- cydata_cleaned%>%
  filter(cydata_cleaned$start_station_name!="DIVVY CASSETTE REPAIR MOBILE STATION" & cydata_cleaned$end_station_name!="DIVVY CASSETTE REPAIR MOBILE STATION")

cydata_cleaned <- cydata_cleaned%>%
  filter(cydata_cleaned$start_station_name!="WATSON TESTING - DIVVY" & cydata_cleaned$end_station_name!="WATSON TESTING - DIVVY")

cydata_cleaned <- cydata_cleaned%>%
  filter(cydata_cleaned$start_station_name!="WEST CHI-WATSON" & cydata_cleaned$end_station_name!="WEST CHI-WATSON")

#### Checking the data after the wrangling.

In [ ]:
## FINAL DATA CHECK
str(cydata_cleaned)
head(cydata_cleaned)
dim(cydata_cleaned)
summary(cydata_cleaned)

#### Saving the final dataframe in a csv file so that we can import it for further use.

In [ ]:
## SAVE CYDATA_CLEANED DATATABLE TO EXCEL FILE
fwrite(cydata_cleaned,"cydata.csv")

### ANALYSE DATA 

#### Ordering the categories of 'day_of_week' into levels so that when we analyse data by that column, we have a consistent order of output in which they display.

In [ ]:
## ORDER DAY OF WEEK IN A ROW FOR CONSISTENCY
cydata_cleaned$day_of_week <- ordered(cydata_cleaned$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

#### Calculating the mean of 'ride_length' by various ways.

In [ ]:
## MEAN OF RIDE LENGTH
as.numeric(mean(cydata_cleaned$ride_length))


## MEAN FOR MEMBER USERS AND CASUAL USERS USING AGGREGATE FUNCTION
aggregate(cydata_cleaned$ride_length,by=list(cydata_cleaned$member_casual),FUN=mean)

## MEAN OF RIDE LENGTH OF MEMBER AND CASUAL USERS FOR VARIOUS DAYS OF WEEK
aggregate(cydata_cleaned$ride_length,by=list(cydata_cleaned$member_casual,cydata_cleaned$day_of_week),FUN=mean)

## MEAN OF RIDE LENGTH FOR DAYS OF WEEK
aggregate(cydata_cleaned$ride_length,by=list(cydata_cleaned$day_of_week),FUN=mean)

#### Calculating the total rides per day of week.

In [ ]:
## NUMBER OF RIDES PER DAY OF WEEK
aggregate(cydata_cleaned$ride_id,by=list(cydata_cleaned$day_of_week),FUN=length)

#### Calculating the maximum ride length in seconds, minutes, hours and days.

In [ ]:
## MAX RIDE LENGTH IN SECONDS
as.numeric(max(cydata_cleaned$ride_length))

## MAX RIDE LENGTH IN MINUTES
as.numeric(max(cydata_cleaned$ride_length))/60

## MAX RIDE LENGTH IN HOURS
as.numeric(max(cydata_cleaned$ride_length)/(60*60))

## MAX RIDE LENGTH IN DAYS
as.numeric(max(cydata_cleaned$ride_length)/(24*60*60))

#### Calculating the day with the most rides.

In [ ]:
## MODE OF WEEKDAY, I.E THE MOST REOCCURING WEEKDAY
mode=mfv(cydata_cleaned$day_of_week)
mode

#### Calculating the median of ride length

In [ ]:
## MEDIAN OF RIDE_LENGTH
as.numeric(median(cydata_cleaned$ride_length))

#### Calculating the number of rides and average duration by member type and weekday.

In [ ]:
## NUMBER OF RIDES AND AVERAGE DURATION GROUPED BY MEMBER_CASUAL AND WEEKDAY
cydata_cleaned %>%
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% 
  summarise(number_of_rides = n(),average_duration = mean(ride_length)) %>% 
  arrange(member_casual, weekday)

### VISUALIZATION

#### USING GGPLOT FOR A QUICK OVERVIEW

#### 1. Number of rides for each rider type, for each day of week

In [ ]:
## NUMBER OF RIDES FOR MEMBER & CASUAL USERS FOR EACH DAY OF WEEK
cydata_cleaned %>%
  mutate(weekday = wday(started_at, label = TRUE)) %>%
  group_by(member_casual, weekday) %>%
  summarise(number_of_rides = n(),average_duration = mean(ride_length)) %>%
  arrange(member_casual, weekday) %>%
  ggplot(aes(x = weekday, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge")

#### 2. Average ride length for each rider type, for each day of week.

In [ ]:
## AVERAGE DURATION OF RIDE LENGTH FOR MEMBER & CASUAL USERS FOR EACH DAY OF WEEK
cydata_cleaned %>%
  mutate(weekday = wday(started_at, label = TRUE)) %>%
  group_by(member_casual, weekday) %>%
  summarise(number_of_rides = n(),average_duration = mean(ride_length)) %>%
  arrange(member_casual, weekday) %>%
  ggplot(aes(x = weekday, y = average_duration, fill = member_casual)) +
  geom_col(position = "dodge")